In [1]:
from sklearn.model_selection import cross_val_score, cross_val_predict 
from sklearn.metrics import confusion_matrix
from sklearn.preprocessing import OrdinalEncoder
from sklearn.tree import DecisionTreeClassifier
import pandas as pd
import numpy as np
import itertools
from preprocess import clean_dataframe

In [22]:
raw_data = pd.read_csv('Murder_Data.zip', index_col=0, compression='zip')
cleaned_data = clean_dataframe(raw_data)

/Users/thomas/opt/anaconda3/lib/python3.8/site-packages/IPython/core/interactiveshell.py:3165: DtypeWarning: Columns (9) have mixed types.Specify dtype option on import or set low_memory=False.
  has_raised = await self.run_ast_nodes(code_ast.body, cell_name,


In [65]:
input_features = ['County', 'State', 'Area', 'VicAge', 'VicSex', 'VicRace', 'VicEthnic', 'VicCount', 'Weapon', 'Subcircum', 'Agency', 'Agentype', 'Circumstance', 'Homicide']
output_features = ['OffAge', 'OffSex', 'OffRace', 'OffEthnic', 'OffCount']

def tree_analysis(X: pd.core.frame.DataFrame, y: pd.core.frame.DataFrame, n_crossval: int = 10, criterion: str = 'entropy', scoring = 'balanced_accuracy'):
    assert criterion in ['entropy', 'gini'], 'invalid choice of criterion. Needs to be entropy or gini.'
    classifier = DecisionTreeClassifier(criterion = criterion)
    y_pred = cross_val_predict(classifier, X = X, y = y, cv = n_crossval)
    cross_val_scores = cross_val_score(classifier, X = X, y = y, cv = n_crossval, scoring = scoring)
    #conf_matrix = confusion_matrix(y, y_pred)
    return(np.mean(cross_val_scores), np.std(cross_val_scores))#, conf_matrix)

In [61]:
X = cleaned_data[input_features]
y = cleaned_data[output_features]
y['OffAge'].replace(to_replace='Unknown', value = 999, inplace = True)
non_numerics = X[['County', 'State', 'Area', 'VicSex', 'VicRace', 'VicEthnic', 'Weapon', 'Subcircum', 'Agency', 'Agentype', 'Circumstance', 'Homicide']]

/Users/thomas/opt/anaconda3/lib/python3.8/site-packages/pandas/core/series.py:4509: SettingWithCopyWarning: 
A value is trying to be set on a copy of a slice from a DataFrame

See the caveats in the documentation: https://pandas.pydata.org/pandas-docs/stable/user_guide/indexing.html#returning-a-view-versus-a-copy
  return super().replace(


In [62]:
non_numerics = pd.DataFrame(OrdinalEncoder().fit_transform(non_numerics), columns = ['County', 'State', 'Area', 'VicSex', 'VicRace', 'VicEthnic', 'Weapon', 'Subcircum', 'Agency', 'Agentype', 'Circumstance', 'Homicide'])
non_numerics = non_numerics.reset_index()
numerics = X[['VicAge', 'VicCount']]
numerics = numerics.reset_index()
X_ordinal = pd.concat([numerics, non_numerics], axis=1)

In [63]:
X_ordinal.drop('ID', axis=1, inplace=True)
X_ordinal['VicAge'].replace(to_replace='Unknown', value = 999, inplace = True)

In [73]:
for target in output_features:
    mean, std = tree_analysis(X = X_ordinal, y = y[target], n_crossval=5, scoring='balanced_accuracy')
    print(f'Predicting {target}: mean accuracy {mean:.6f} with standard deviation {std:.6f}')

Predicting OffAge: mean accuracy 0.022677 with standard deviation 0.002267
Predicting OffSex: mean accuracy 0.530475 with standard deviation 0.025911
Predicting OffRace: mean accuracy 0.183859 with standard deviation 0.085081
Predicting OffEthnic: mean accuracy 0.240171 with standard deviation 0.121707


/Users/thomas/opt/anaconda3/lib/python3.8/site-packages/sklearn/model_selection/_split.py:666: UserWarning: The least populated class in y has only 1 members, which is less than n_splits=5.
  warnings.warn(("The least populated class in y has only %d"
/Users/thomas/opt/anaconda3/lib/python3.8/site-packages/sklearn/model_selection/_split.py:666: UserWarning: The least populated class in y has only 1 members, which is less than n_splits=5.
  warnings.warn(("The least populated class in y has only %d"
/Users/thomas/opt/anaconda3/lib/python3.8/site-packages/sklearn/metrics/_classification.py:1850: UserWarning: y_pred contains classes not in y_true
  warnings.warn('y_pred contains classes not in y_true')
/Users/thomas/opt/anaconda3/lib/python3.8/site-packages/sklearn/metrics/_classification.py:1850: UserWarning: y_pred contains classes not in y_true
  warnings.warn('y_pred contains classes not in y_true')
/Users/thomas/opt/anaconda3/lib/python3.8/site-packages/sklearn/metrics/_classificatio

Predicting OffCount: mean accuracy 0.086959 with standard deviation 0.018451


In [ ]:
# trying if training with dummies is feasible
X = pd.get_dummies(X, columns=['County', 'State', 'Area', 'VicSex', 'VicRace', 'VicEthnic', 'VicCount', 'Weapon', 'Subcircum', 'Agency', 'Agentype', 'Circumstance', 'Homicide'])
mean, std, conf_matrix = tree_analysis(X = X, y = y)
# result: training this one decision tree takes 12 hours with dummy variables